# Using book ratings to predict similar books to a specific book

Starting with a dataset of Amazon book reviews that has about 3 mil rows. I created a smaller dataset by pulling out only the columns I needed, plus dropping rows that have a null value. 
```
# rating_cols = ['Id', 'User_id', 'Title', 'review/score']
# reviews = pd.read_csv("../datasets/Books_rating.csv", names=rating_cols, usecols=range(4), header=0)

# reviews.dropna(inplace= True)
# reviews.to_csv('../datasets/Book_ratings_clean.csv')
```
to sample a subset randomly `reviews = reviews.sample(frac=0.5, random_state=1).reset_index(drop=True)` 
I didn't need to do this, but it would improve the speed (would also leave out some hits though..)

Next, I updated the column names and imported it into a postgres table so I could easily search it. I'm sure it's just as fast to do it here, but I'm more comfortable with SQL at the moment. 

In [2]:
import pandas as pd
# using that cleaner dataset
ratings = pd.read_csv("../datasets/Book_ratings_clean.csv")
ratings.head()

,id,book_id,title,rating,user_id
0,10,0829814000,Wonderful Worship in Smaller Churches,19.40,AZ0IOBU20TBOP
1,11,0829814000,Wonderful Worship in Smaller Churches,19.40,A373VVEU6Z9M0N
2,12,0829814000,Wonderful Worship in Smaller Churches,19.40,AGKGOH65VTRR4
3,13,0829814000,Wonderful Worship in Smaller Churches,19.40,A3OQWLU31BU1Y
4,14,0595344550,Whispers of the Wicked Saints,10.95,A3Q12RK71N74LB


many books only have 1 review, aggregating the rating counts and mean to make it easier to work with 

In [3]:
import numpy as np
bookReviews = ratings.groupby('title').agg({'rating': [np.size, np.mean]})
bookReviews.head()

rating       
                                                     size   mean
title                                                           
"A Truthful Impression of the Country": British...      1  70.27
"Beauty Shop-Philly Style"                              2  13.95
"Civilizing" Rio: Reform and Resistance in a Br...      1  32.95
"Come and See" Kids: The Life of Jesus (Catholi...      2   9.65
"Come to Me"                                            1  10.19

Too many rows for my laptop to churn through, so dropping out books with less than 10 reviews

In [4]:
popularBooks = bookReviews['rating']['size'] >= 10
bookReviews[popularBooks].sort_values([('rating', 'mean')], ascending=False)[:15]

rating        
                                                     size    mean
title                                                            
Starkissed                                             35  730.00
The Collected Works of J.G. Frazer: The Golden ...     49  630.00
Atlas of Hematology                                    10  455.00
Social Theory and Methodology: From Max Weber: ...     11  410.00
Richard II (Shakespeare: The Critical Tradition)       13  360.00
Vietnamese, Comprehensive: Learn to Speak and U...     10  345.00
Polish, Comprehensive: Learn to Speak and Under...     12  287.38
Spanish III, Comprehensive: Learn to Speak and ...     28  287.38
Italian II, Second Edition: Compehensive Compac...     11  287.38
Czech, Comprehensive: Learn to Speak and Unders...     12  287.38
The Oxford Encyclopedia of Food and Drink in Am...     10  283.50
Western Civilization: Combined Volume                  10  262.95
French III, Comprehensive: Learn to Speak and U...     17  241.50
Spanish II, 3rd Ed. (Compr.) [CD]                      24  234.05
Essentials of Business Communication                   17  233.95

Using the pivot_table function on a DataFrame will construct a user / book rating matrix. This process takes about 5 mins to run on my 2020 MBP, watch the activity monitor to see the process. 
Next, I cast this back to a dataframe to work with, then joined it to the original dataframe to get ratings and user_id's. Left join here leaves out books from the original set that aren't in the popular list (10 or more reviews). 

In [5]:
popularBooksDF = popularBooks.to_frame()
# join this back with the original ratings df to get the ratings and user_ids
popularBooksDF = popularBooksDF.join(ratings.set_index('title'), on='title', how='left')
popularBooksDF.head()

,size,id,book_id,rating,user_id
title,,,,,
"""A Truthful Impression of the Country"": British and American Travel Writing in China, 1880-1949",False,2588738,0472111973,70.27,A912C7977MO6O
"""Beauty Shop-Philly Style""",False,1274800,1420859323,13.95,A23JCW11WWEQCQ
"""Beauty Shop-Philly Style""",False,1274801,1420859323,13.95,A1Y2KST29TFAPF
"""Civilizing"" Rio: Reform and Resistance in a Brazilian City 1889-1930",False,1036878,027102870X,32.95,A276Y65EABFM69
"""Come and See"" Kids: The Life of Jesus (Catholic Bible Study for Children)",False,755357,1931018286,9.65,A1JM25M2PSVPXE


In [ ]:
bookReviewsPTable = popularBooksDF.pivot_table(index=['user_id'],columns=['title'],values='rating')
bookReviewsPTable.head()

/Users/vanessaholland/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/reshape.py:134: PerformanceWarning: The following operation may generate 14469434325 cells in the resulting pandas object.
  warnings.warn(


In [ ]:
specificBookRatings = bookReviewsPTable["The Picture of Dorian Gray	"]
specificBookRatings.count()

from the frank kane course: Pandas' corrwith function makes it really easy to compute the pairwise correlation of the book's vector of user rating with every other book! After that, we'll drop any results that have no data, and construct a new DataFrame of books and their correlation score (similarity) to the specific one from above:

In [ ]:
similarBooks = bookReviewsPTable.corrwith(specificBookRatings)
similarBooks = similarBooks.dropna()
df = pd.DataFrame(similarBooks)
df.head(10)

ToDo- resolve that error. for now it's safe.

In [ ]:
similarBooks.sort_values(ascending=False)

Ten might be too low.. may play around with it. 

In [ ]:
df = bookReviews[popularBooks].join(pd.DataFrame(similarBooks, columns=['similarity']))

In [ ]:
df.head()

In [ ]:
df.sort_values(['similarity'], ascending=False)[:15]

Follow-ups: remove original book from results, and learn how to work with big data more efficiently. Specifically Spark.

Results it spits out when the kernal doesn't die: 
```
Alice's Adventures in Wonderland and Through the Looking Glass (Classic Collection (Brilliance Audio))    1.000
The Picture of Dorian Gray                                                                                1.000
The Red Badge of Courage (Lake Illustrated Classics, Collection 1)                                        1.000
Wuthering Heights                                                                                         1.000
the Picture of Dorian Gray                                                                                1.000
A Christmas Carol (Classic Fiction)                                                                       1.000
A Christmas Carol, in Prose: Being a Ghost Story of Christmas (Collected Works of Charles Dickens)        1.000
The Picture of Dorian Gray (The Classic Collection)                                                       0.998
```